In [47]:
import pandas as pd
import os
from nltk.corpus import stopwords

ModuleNotFoundError: No module named 'nltk'

In [8]:
os.getcwd()

'/home/eetil1818/code/nghia95/fake-data-detector/notebooks'

In [52]:
path = "/home/eetil1818/code/nghia95/fake-data-detector/raw_data/1k_sampled_dataset.csv"
data = pd.read_csv(path)
data.head()

,text,source,prompt_id,text_length,word_count
0,The Philosophy and Ethics of Transhumanism\n\n...,GPT-3.5,1920,2558,394
1,Crime-tracking app Citizen is launching its ow...,Flan-T5-XXL,0,378,62
2,The court in Novorossiysk gave two of the danc...,GLM-130B,0,621,109
3,"then drops the drumsticks, poses, then walks o...",GPT-J,0,513,90
4,On tally went to the beach. She found a sand d...,GPT-J,0,4984,846


In [62]:
df = data.copy()
df["source"] = df["source"].apply(lambda x: "AI" if x != "Human" else x)

In [63]:
df["source"].value_counts(normalize=True)

source
Human    0.559
AI       0.441
Name: proportion, dtype: float64

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         1000 non-null   object
 1   source       1000 non-null   object
 2   prompt_id    1000 non-null   int64 
 3   text_length  1000 non-null   int64 
 4   word_count   1000 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 39.2+ KB


In [65]:
df.drop(columns="prompt_id", inplace=True)
df.head()

,text,source,text_length,word_count
0,The Philosophy and Ethics of Transhumanism\n\n...,AI,2558,394
1,Crime-tracking app Citizen is launching its ow...,AI,378,62
2,The court in Novorossiysk gave two of the danc...,AI,621,109
3,"then drops the drumsticks, poses, then walks o...",AI,513,90
4,On tally went to the beach. She found a sand d...,AI,4984,846


In [75]:
# Check if text column contains numbers and special characters, do we want to clean them?

df["contains_numbers"] = df["text"].str.contains(r"\d", regex=True)
df["contains_special_chars"] = df["text"].str.contains(r"[^\w\s]", regex=True)

df.head()

,text,source,text_length,word_count,len,contains_numbers,contains_special_chars
0,The Philosophy and Ethics of Transhumanism\n\n...,AI,2558,394,2558,False,True
1,Crime-tracking app Citizen is launching its ow...,AI,378,62,378,True,True
2,The court in Novorossiysk gave two of the danc...,AI,621,109,621,True,True
3,"then drops the drumsticks, poses, then walks o...",AI,513,90,513,True,True
4,On tally went to the beach. She found a sand d...,AI,4984,846,4984,False,True


In [ ]:
# for cleaning later, please ignore for now

#def clean_text():
    #text = text.str.lower() # turn to all lowercase
    #text = text.str.strip() # remove whitespace